In [130]:
import pandas as pd

data = pd.read_pickle(r'C:/Users/Kepa/Downloads/data_to_fit_CNNIIC_iic_burden_smooth+spike_rate.pickle')

In [131]:
type(data)

dict

In [132]:
data.keys()

dict_keys(['W', 'window_start_ids', 'D', 'Ddose', 'Dname', 'Pobs', 'Pname', 'C', 'Cname', 'Y', 'Yname', 'cluster', 'sids', 'pseudoMRNs'])

In [133]:
print(data['W'])

300


In [166]:
print(data['sids'])

['sid2' 'sid3' 'sid4' ... 'sid1991' 'sid1997' 'sid1999']


In [283]:
print(data['Dname'])

['lacosamide', 'levetiracetam', 'midazolam', 'pentobarbital', 'phenobarbital', 'propofol', 'valproate']


In [134]:
data['Pobs']['iic_burden_smooth'][0]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.95666667, 0.6862069 , 0.52333333,
       0.45535714,        nan, 0.        , 0.16549296, 0.62      ,
       0.05      , 0.43928571, 0.08333333, 0.41935484, 0.37      ,
       0.65116279, 0.90774908, 1.        , 0.96333333, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 0.75      , 0.41454545, 0.        ,
       0.01666667, 0.02666667, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.01779359, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05902778, 0.6137931 ,
       0.        , 0.02479339, 0.        , 0.21299639, 0.     

In [135]:
data['D'][0].shape

(197, 7)

In [138]:
import numpy as np
import scipy as sp
from scipy.optimize import least_squares

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [271]:
def PD_objective(beta,E,D):
    #beta[2*i] is hill coeffiecint for drug i
    #beta[2*i+1] is ED_50 for drug i 
    #i = 0:6
    return sum((1 - (D[:,0]**beta[0] )/(D[:,0]**beta[0] + beta[1]**beta[0]) +
               (D[:,1]**beta[2] )/(D[:,1]**beta[2] + beta[3]**beta[2])+ 
               (D[:,2]**beta[4] )/(D[:,2]**beta[4] + beta[5]**beta[4])+
               (D[:,3]**beta[6] )/(D[:,3]**beta[6] + beta[7]**beta[6])+
               (D[:,4]**beta[8] )/(D[:,4]**beta[8] + beta[9]**beta[8])+
               (D[:,5]**beta[10] )/(D[:,5]**beta[10] + beta[11]**beta[10])+ 
               (D[:,6]**beta[12] )/(D[:,6]**beta[12] + beta[13]**beta[12])) - E)

final_df  = np.full([1310,14], np.nan)
final_df


for i in range(0,1310 ):
    E = object['Pobs']['iic_burden_smooth'][i]
    E = E.reshape(len(E),1)
    D = object['D'][i]
    idx = abs(D).sum(axis = 0) > 0
    n_drugs = sum(idx)
    
    df = np.concatenate((E,D), axis = 1)
    df = df[~np.isnan(df).any(axis=1)]
   
    if(n_drugs ==0):
        final_df[i,:] = np.repeat(np.nan, 14)
    else:
        x0_PD = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1])
        #uncomment for unconstrainted
        #result = least_squares(PD_objective, x0_PD, args=[df[:,0], df[:,1:]]  )
        
        #uncomment for constrained
        result = least_squares(PD_objective, x0_PD, args=[df[:,0], df[:,1:]] ,bounds= ( [0,0,0,0,0,0,0,0,0,0,0,0,0,0], np.inf ) )
        
        missing_idx = np.concatenate( ( 2*np.where(idx == False)[0], 2*np.where(idx == False)[0] + 1))
        final_df[i,:] = result['x']
        final_df[i,missing_idx] = np.nan
        
    
        


    

<ipython-input-271-cdf8a595ee5a>:10: RuntimeWarning: overflow encountered in power
  (D[:,1]**beta[2] )/(D[:,1]**beta[2] + beta[3]**beta[2])+
<ipython-input-271-cdf8a595ee5a>:10: RuntimeWarning: overflow encountered in double_scalars
  (D[:,1]**beta[2] )/(D[:,1]**beta[2] + beta[3]**beta[2])+
<ipython-input-271-cdf8a595ee5a>:10: RuntimeWarning: invalid value encountered in true_divide
  (D[:,1]**beta[2] )/(D[:,1]**beta[2] + beta[3]**beta[2])+


In [284]:
final_df2 = pd.DataFrame(final_df, columns = ['lacosamide_Hill', 'lacosamide_50',
                                              'levetiracetam_Hill', 'levetiracetam_50',
                                              'midazolam_Hill', 'midazolam_50',
                                              'pentobarbital_Hill', 'pentobarbital_50',
                                              'phenobarbital_Hill', 'phenobarbital_50',
                                              'propofol_Hill', 'propofol_50',
                                              'valproate_Hill', 'valproate_50' ])
final_df2['sids'] = data['sids']

In [285]:
final_df2

,lacosamide_Hill,lacosamide_50,levetiracetam_Hill,levetiracetam_50,midazolam_Hill,midazolam_50,pentobarbital_Hill,pentobarbital_50,phenobarbital_Hill,phenobarbital_50,propofol_Hill,propofol_50,valproate_Hill,valproate_50,sids
0,NaN,NaN,1.887647e-16,57.009882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sid2
1,NaN,NaN,4.900035e-16,86.947726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sid3
2,NaN,NaN,7.348216e-13,18.497718,NaN,NaN,NaN,NaN,NaN,NaN,23.802689,34.336808,NaN,NaN,sid4
3,7.738566,0.07188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.346894,33.508543,NaN,NaN,sid5
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sid6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,NaN,1.788990e-18,57.009897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sid1989
1306,NaN,NaN,2.554595e-15,57.010982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sid1990
1307,NaN,NaN,1.062644e-18,50.167064,NaN,NaN,NaN,NaN,NaN,NaN,37.790463,49.312709,NaN,NaN,sid1991
1308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sid1997


In [286]:
final_df2.describe()

,lacosamide_Hill,lacosamide_50,levetiracetam_Hill,levetiracetam_50,midazolam_Hill,midazolam_50,pentobarbital_Hill,pentobarbital_50,phenobarbital_Hill,phenobarbital_50,propofol_Hill,propofol_50,valproate_Hill,valproate_50
count,107.000000,107.000000,8.250000e+02,825.000000,1.340000e+02,134.000000,7.000000e+00,7.000000,3.700000e+01,37.000000,6.470000e+02,647.000000,5.400000e+01,54.000000
mean,9.349548,0.569742,1.079774e+01,50.999942,1.466719e+01,12.513297,3.757695e-13,29.740157,1.752365e+01,31.258455,2.295691e+01,28.551287,9.440983e+00,50.069078
std,9.949867,1.871627,2.138444e+01,52.099214,1.010286e+01,10.072970,9.840513e-13,24.361598,2.366728e+01,25.734400,7.357858e+00,9.232482,2.052966e+01,41.120306
min,1.317355,0.000002,1.058491e-63,0.528473,1.699406e-16,1.000000,3.066627e-42,1.874214,4.436900e-32,4.739777,2.638129e-34,1.263921,9.678769e-39,1.958386
25%,4.849894,0.008305,9.476866e-19,15.773046,8.134285e+00,5.961924,6.694654e-21,12.966968,5.216445e-17,17.121785,1.888105e+01,22.303937,6.680590e-20,21.741173
50%,5.911238,0.042152,1.612405e-15,33.884607,1.173334e+01,9.102567,1.745925e-16,32.703044,1.451797e+01,23.436686,2.236931e+01,28.207228,1.963414e-15,40.078502
75%,8.118421,0.134788,3.507637e-11,65.110131,1.767663e+01,16.551239,1.144383e-14,36.924827,2.472321e+01,38.371934,2.544668e+01,34.125308,2.453466e-13,71.720209
max,61.417489,13.970284,1.378307e+02,367.642168,6.173090e+01,56.504408,2.607324e-12,73.820250,1.199171e+02,116.104960,7.012654e+01,74.746587,6.847853e+01,184.402212


In [282]:
final_df2.to_csv('PD_Parameters_result.csv')